In [1]:
# !pip install wandb
import wandb

In [2]:
from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision.transforms import transforms


# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 (compatible with the CNN input size)
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize image tensors
])

# Define the path to the iNaturalist dataset directory
data_dir = '/kaggle/input/naturist-data/nature_12K/inaturalist_12K/train'

# Load the iNaturalist dataset using ImageFolder and apply transformations
train_dataset = ImageFolder(root=data_dir, transform=transform)

# Split the dataset into training and validation indices
train_indices, val_indices = train_test_split(list(range(len(train_dataset))), test_size=0.2, random_state=42)

# Create DataLoader instances for training and validation sets
train_sampler = SubsetRandomSampler(train_indices)
train_loader = DataLoader(train_dataset, batch_size=64, sampler=train_sampler)

val_sampler = SubsetRandomSampler(val_indices)
val_loader = DataLoader(train_dataset, batch_size=64, sampler=val_sampler)

from torch.utils.data.sampler import SubsetRandomSampler

# Define the path to the iNaturalist dataset directory
test_data_dir = '/kaggle/input/naturist-data/nature_12K/inaturalist_12K/val'

# Define transformations with data augmentation
transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # ImageNet normalization
])

# Load the iNaturalist dataset using ImageFolder and apply transformations
test_dataset = ImageFolder(root=test_data_dir, transform=transform)

test_loader = DataLoader(test_dataset, batch_size=64)


# Example usage:
for images, labels in test_loader:
    print("Test batch shapes:", images.shape, labels.shape)
    break

# Example usage:
for images, labels in train_loader:
    print("Training batch shapes:", images.shape, labels.shape)
    break

for images, labels in val_loader:
    print("Validation batch shapes:", images.shape, labels.shape)
    break

Test batch shapes: torch.Size([64, 3, 224, 224]) torch.Size([64])
Training batch shapes: torch.Size([64, 3, 224, 224]) torch.Size([64])
Validation batch shapes: torch.Size([64, 3, 224, 224]) torch.Size([64])


In [3]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [14]:
from tqdm import tqdm

def train_and_evaluate(model, train_loader, val_loader, criterion, optimizer, num_epochs=1):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     device = 'cuda'
#     model.to(device)
    for epoch in range(num_epochs):
        model.train()  # Set model to train mode
        running_loss = 0.0
        train_loss = 0.0
        correct = 0
        total = 0
        for ind, (images, labels) in enumerate(tqdm(train_loader)):
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            optimizer.zero_grad()  # Zero the parameter gradients
            outputs = model(images)  # Forward pass
            loss = criterion(outputs, labels)  # Compute loss
            train_loss = train_loss + loss
            loss.backward()  # Backward pass
            optimizer.step()  # Optimize
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        train_loss = train_loss / len(train_loader.dataset)
        train_accuracy = correct / total

        # Validation loop
        model.eval()  # Set model to evaluation mode
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)  # Move data to GPU
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        val_loss = val_loss / len(val_loader.dataset)
        val_accuracy = correct / total

        # Print epoch statistics
        print(f'Epoch {epoch + 1}/{num_epochs}: Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}')

        # Log to WandB
        wandb.log({
            'epoch': epoch + 1,
            'train_loss': train_loss,
            'train_accuracy': train_accuracy,
            'val_loss': val_loss,
            'val_accuracy': val_accuracy
        })


In [18]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torch.optim import Adam
import wandb

# Define your ResNet50-based model
class ResNet50Model(nn.Module):
    def __init__(self, num_classes, freeze_percentage=0.5):
        super(ResNet50Model, self).__init__()
        self.resnet50 = models.resnet50(pretrained=True)
        self.freeze_layers(freeze_percentage)  # Freeze a percentage of layers

        # Modify the final fully connected layer according to the number of classes
        num_features = self.resnet50.fc.in_features
        self.resnet50.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        return self.resnet50(x)

    def freeze_layers(self, freeze_percentage):
        # Get the total number of layers
        total_layers = sum(1 for _ in self.resnet50.parameters())

        # Calculate the number of layers to freeze
        num_layers_to_freeze = int(total_layers * freeze_percentage)

        # Freeze the layers
        frozen_layers = 0
        for param in self.resnet50.parameters():
            param.requires_grad = False
            frozen_layers += 1
            if frozen_layers >= num_layers_to_freeze:
                break

In [16]:
# model = ResNet50Model(num_classes=10, freeze_percentage=0.5)
# print(model)

In [17]:
import wandb
wandb.login(key='12c0b23d6865ce943b48c8ea1451c9b2d3aedf60')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [21]:
# Define the parameters for hyperparameter tuning
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'num_epochs': {
            'values': [1, 2, 3]  # Fixed typo in parameter name
        },
        'learning_rate': {
            'values': [1e-3, 1e-4, 1e-5]
        },
        'freeze_percentage': {
            'values': [0.30, 0.50, 0.80]
        },
        'optimizer': {
            'values': ['adam', 'SGD']
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project='Assignment_2')

Create sweep with ID: aixat1jw
Sweep URL: https://wandb.ai/lokendrakumar/Assignment_2/sweeps/aixat1jw


In [22]:
# Define the main function
def main():
    # Initialize a new wandb run
    with wandb.init() as run:
        # Construct run name based on hyperparameters
        run_name = f"ResNet50-Epochs-{wandb.config.num_epochs}-LR-{wandb.config.learning_rate}-FreezePercent-{wandb.config.freeze_percentage}"
        wandb.run.name = run_name

        # Define data transformations
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

        # Load datasets
        train_dataset = ImageFolder(root=data_dir, transform=transform)
        train_indices, val_indices = train_test_split(list(range(len(train_dataset))), test_size=0.2, random_state=42)
        train_sampler = SubsetRandomSampler(train_indices)
        train_loader = DataLoader(train_dataset, batch_size=32, sampler=train_sampler)
        val_sampler = SubsetRandomSampler(val_indices)
        val_loader = DataLoader(train_dataset, batch_size=32, sampler=val_sampler)

        # Create ResNet50 model instance
        model = ResNet50Model(num_classes=10)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)

        # Define criterion and optimizer
        criterion = nn.CrossEntropyLoss()
        # optimizer = Adam(model.parameters(), lr=wandb.config.learning_rate)

        # Define optimizer based on configuration
        if wandb.config.optimizer == 'adam':
            optimizer = torch.optim.Adam(model.parameters(), lr=wandb.config.learning_rate)
        elif wandb.config.optimizer == 'SGD':
            optimizer = torch.optim.SGD(model.parameters(), lr=wandb.config.learning_rate, momentum=0.9)

        # Train and evaluate the model
        train_and_evaluate(model, train_loader, val_loader, criterion, optimizer, num_epochs=wandb.config.num_epochs)

# Run the sweep to perform experiments
wandb.agent(sweep_id, function=main, count=10)

wandb: Agent Starting Run: gggnux58 with config:
wandb: 	freeze_percentage: 0.8
wandb: 	learning_rate: 0.0001
wandb: 	num_epochs: 3
wandb: 	optimizer: SGD


100%|██████████| 250/250 [02:16<00:00,  1.83it/s]


Epoch 1/3: Train Loss: 0.0493, Train Acc: 0.4321, Val Loss: 0.3234, Val Acc: 0.6205


100%|██████████| 250/250 [02:11<00:00,  1.90it/s]


Epoch 2/3: Train Loss: 0.0351, Train Acc: 0.6585, Val Loss: 0.2334, Val Acc: 0.6990


...

epoch,▁▅█
train_accuracy,▁▅█
train_loss,█▄▁
val_accuracy,▁██
val_loss,█▁▃
epoch,3
train_accuracy,0.72197
train_loss,0.02025
val_accuracy,0.641
val_loss,0.23175


wandb: Agent Starting Run: oiqy1046 with config:
wandb: 	freeze_percentage: 0.3
wandb: 	learning_rate: 0.001
wandb: 	num_epochs: 1
wandb: 	optimizer: adam


100%|██████████| 250/250 [02:16<00:00,  1.83it/s]


Epoch 1/1: Train Loss: 0.0386, Train Acc: 0.4634, Val Loss: 0.2500, Val Acc: 0.5860


epoch,▁
train_accuracy,▁
train_loss,▁
val_accuracy,▁
val_loss,▁
epoch,1
train_accuracy,0.46343
train_loss,0.03863
val_accuracy,0.586
val_loss,0.25001
